In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import random
import warnings
from itertools import product

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
from tqdm.notebook import tqdm
from sklearn.exceptions import ConvergenceWarning

try:
    from myst_nb import glue
except ImportError:
    glue = lambda *args, **kwargs: _
# 一部の警告を無視
warnings.simplefilter("ignore", ConvergenceWarning)
warnings.simplefilter("ignore", FutureWarning)
os.environ["PYTHONWARNINGS"] = "ignore"

In [2]:
import itertools

K=40
max_value = 2**K
max_bit_count = 4
count = 0

for i in range(max_bit_count + 1):
    combinations = list(itertools.combinations(range(K), i))
    for comb in combinations:
        num = sum(1 << j for j in comb)
        if num <= max_value:
            count += 1
            print(num)

print(count)

0
1
2
4
8
16
32
64
128
256
512
1024
2048
4096
8192
16384
32768
65536
131072
262144
524288
1048576
2097152
4194304
8388608
16777216
33554432
67108864
134217728
268435456
536870912
1073741824
2147483648
4294967296
8589934592
17179869184
34359738368
68719476736
137438953472
274877906944
549755813888
3
5
9
17
33
65
129
257
513
1025
2049
4097
8193
16385
32769
65537
131073
262145
524289
1048577
2097153
4194305
8388609
16777217
33554433
67108865
134217729
268435457
536870913
1073741825
2147483649
4294967297
8589934593
17179869185
34359738369
68719476737
137438953473
274877906945
549755813889
6
10
18
34
66
130
258
514
1026
2050
4098
8194
16386
32770
65538
131074
262146
524290
1048578
2097154
4194306
8388610
16777218
33554434
67108866
134217730
268435458
536870914
1073741826
2147483650
4294967298
8589934594
17179869186
34359738370
68719476738
137438953474
274877906946
549755813890
12
20
36
68
132
260
516
1028
2052
4100
8196
16388
32772
65540
131076
262148
524292
1048580
2097156
4194308
8388612


In [3]:
Xs=[]
Ys=[]
def load_data(data_name,N,M,K,pixel):
    for id in range(K):
        # id=random.randint(0,K-1)
        img=cv2.imread(f'./training_data/{data_name}/numbers_{id}.png')
        numbers=np.load(f'./training_data/{data_name}/numbers_{id}.npy')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # plt.imshow(img, cmap='gray')
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # plt.imshow(img, cmap='gray')
        # print(img.shape)
        img_rs=cv2.resize(gray,(pixel*M,pixel*N),interpolation=cv2.INTER_AREA)
    # img_rs=cv2.bitwise_not(img_rs)
    # print(img_rs.shape,i)

        for i in range(N):
            for j in range(M):
                digit_square = img_rs[i*pixel:(i+1)*pixel, j*pixel:(j+1)*pixel]
                x_i=digit_square.reshape(1, -1)
                # print(x_i.shape,i,j)
                y=numbers[i,j]
                Xs.append(x_i)
                Ys.append(y)

In [4]:
# 白地のまま学習する

# 画像を読み込む
try:
    train
except:
    train=1
save=1
pixel=20
N,M=50,50
K=2
# data_name='numbers'
# load_data(data_name,N,M,K,pixel)
# load_data('numbers',50,50,10,pixel)
load_data('numbers_v2',10,10,500,pixel)
load_data('numbers_line_3',10,10,500,pixel)
# load_data('numbers_line',50,50,10,pixel)
load_data('numbers_ln_v2',10,10,500,pixel)
load_data('numbers_ln_m',10,10,500,pixel)
# load_data('numbers_line_wb',50,50,10,pixel)
# load_data('numbers_line_wb_light',10,10,100,pixel)
# load_data('numbers_wo0',50,50,10,pixel)
# load_data('numbers_wo0_line',50,50,10,pixel)


        
Xs=pd.DataFrame(np.array(Xs).reshape(-1,pixel*pixel))
Ys=pd.DataFrame(np.array(Ys).reshape(-1,1),columns=['label'])
print(Xs.shape,Ys.shape)
df=pd.concat([Ys,Xs],axis=1)
# df=df.loc[df.iloc[:,0]!=0]
df.shape


(200000, 400) (200000, 1)


(200000, 401)

In [5]:
# i,j=10,5
# plt.imshow(img_rs[i*pixel:(i+1)*pixel, j*pixel:(j+1)*pixel], cmap='gray')


In [6]:
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
X_org, y_org = df.iloc[:,1:], df.iloc[:,0]
X_org = np.array(X_org / 255.0, dtype="double")
y_org = np.array(y_org, dtype="uint8")
X, X_test, y, y_test = model_selection.train_test_split(
    X_org, y_org,test_size=0.2, shuffle=True,random_state=42
)
# use partial data 
# n_samples = 10000  
# X, y = X[:n_samples], y[:n_samples]
# X_test, y_test = X_test[:n_samples], y_test[:n_samples]

# スケーリングパラメータの計算
# scaler = StandardScaler()
# scaler.fit(X)

# # データのスケーリング
# X = scaler.transform(X)
# X_test = scaler.transform(X_test)


In [7]:
model_name="ensemble_MLPC"
# model_name="MLPC_numbers_mix"
clf=pd.read_pickle(f"./pickle/{model_name}_clf.pickle")
scaler=pd.read_pickle(f"./pickle/{model_name}_scaler.pickle")
X=scaler.transform(X)
X_test=scaler.transform(X_test)
X_org=scaler.transform(X_org)
acc_train = 100.0 * clf.score(X, y)
print("logis_acc_train", acc_train)

# テストデータに対する識別精度の計算
acc_test = 100.0 * clf.score(X_test, y_test)
print("logis_acc_test", acc_test)
# confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X_org)
labels=[0,1,2,3,4,5,6,7,8,9]
cm = confusion_matrix(y_org, y_pred,labels=labels)
print(cm)

logis_acc_train 96.599375
logis_acc_test 96.555
[[20059     7     4     4     2    13     9     4     2    11]
 [   56 19342    81   244    80    28     7    51     4    15]
 [   23    89 19334   245    35    87    20   127    52   196]
 [   22   195   365 19251    32   173     3    17    67    94]
 [   13   163    68    39 19215    54   143     5    33    57]
 [   12   103    24   184    28 19009   364     8   179    88]
 [    7    17    52     9   110   196 19317     5   218    56]
 [   21   161   180    45     5    68    10 19188     0   139]
 [    2    10    80    49    67   176   145     7 19102   235]
 [   19    12    44    99    85   291    77    17    76 19364]]


In [8]:
print("Confusion Matrix:")
print("Labels: ", labels)
print("      ", end="")
for label in labels:
    print(f"{label:4d}", end=" ")
print()
for i, row in enumerate(cm):
    print(f"    {i:2d} ", end="")
    for val in row:
        print(f"{val:4d}", end=" ")
    print()

Confusion Matrix:
Labels:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
         0    1    2    3    4    5    6    7    8    9 
     0 20059    7    4    4    2   13    9    4    2   11 
     1   56 19342   81  244   80   28    7   51    4   15 
     2   23   89 19334  245   35   87   20  127   52  196 
     3   22  195  365 19251   32  173    3   17   67   94 
     4   13  163   68   39 19215   54  143    5   33   57 
     5   12  103   24  184   28 19009  364    8  179   88 
     6    7   17   52    9  110  196 19317    5  218   56 
     7   21  161  180   45    5   68   10 19188    0  139 
     8    2   10   80   49   67  176  145    7 19102  235 
     9   19   12   44   99   85  291   77   17   76 19364 


In [17]:
results = []
for i in range(len(labels)):
    col = [row[i] for row in cm]
    top_n=10
    top2 = np.argsort(col)[::-1][:top_n]
    results.append([labels[top2[i]] for i in range(top_n)])

# 結果の出力
for i, col in enumerate(results):
    print(f"{labels[i]}: {col}")

0: [0, 1, 2, 3, 7, 9, 4, 5, 6, 8]
1: [1, 3, 4, 7, 5, 2, 6, 9, 8, 0]
2: [2, 3, 7, 1, 8, 4, 6, 9, 5, 0]
3: [3, 2, 1, 5, 9, 8, 7, 4, 6, 0]
4: [4, 6, 9, 1, 8, 2, 3, 5, 7, 0]
5: [5, 9, 6, 8, 3, 2, 7, 4, 1, 0]
6: [6, 5, 8, 4, 9, 2, 7, 0, 1, 3]
7: [7, 2, 1, 9, 3, 5, 8, 6, 4, 0]
8: [8, 6, 5, 9, 3, 2, 4, 1, 0, 7]
9: [9, 8, 2, 7, 3, 5, 4, 6, 1, 0]


In [10]:
results

[[0, 1, 2],
 [1, 3, 4],
 [2, 3, 7],
 [3, 2, 1],
 [4, 6, 9],
 [5, 9, 6],
 [6, 5, 8],
 [7, 2, 1],
 [8, 6, 5],
 [9, 8, 2]]

In [11]:
for i in range(3**5):
    nums = [(i // 3**j) % 3 for j in range(5)]
    print(nums)

[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[2, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[1, 1, 0, 0, 0]
[2, 1, 0, 0, 0]
[0, 2, 0, 0, 0]
[1, 2, 0, 0, 0]
[2, 2, 0, 0, 0]
[0, 0, 1, 0, 0]
[1, 0, 1, 0, 0]
[2, 0, 1, 0, 0]
[0, 1, 1, 0, 0]
[1, 1, 1, 0, 0]
[2, 1, 1, 0, 0]
[0, 2, 1, 0, 0]
[1, 2, 1, 0, 0]
[2, 2, 1, 0, 0]
[0, 0, 2, 0, 0]
[1, 0, 2, 0, 0]
[2, 0, 2, 0, 0]
[0, 1, 2, 0, 0]
[1, 1, 2, 0, 0]
[2, 1, 2, 0, 0]
[0, 2, 2, 0, 0]
[1, 2, 2, 0, 0]
[2, 2, 2, 0, 0]
[0, 0, 0, 1, 0]
[1, 0, 0, 1, 0]
[2, 0, 0, 1, 0]
[0, 1, 0, 1, 0]
[1, 1, 0, 1, 0]
[2, 1, 0, 1, 0]
[0, 2, 0, 1, 0]
[1, 2, 0, 1, 0]
[2, 2, 0, 1, 0]
[0, 0, 1, 1, 0]
[1, 0, 1, 1, 0]
[2, 0, 1, 1, 0]
[0, 1, 1, 1, 0]
[1, 1, 1, 1, 0]
[2, 1, 1, 1, 0]
[0, 2, 1, 1, 0]
[1, 2, 1, 1, 0]
[2, 2, 1, 1, 0]
[0, 0, 2, 1, 0]
[1, 0, 2, 1, 0]
[2, 0, 2, 1, 0]
[0, 1, 2, 1, 0]
[1, 1, 2, 1, 0]
[2, 1, 2, 1, 0]
[0, 2, 2, 1, 0]
[1, 2, 2, 1, 0]
[2, 2, 2, 1, 0]
[0, 0, 0, 2, 0]
[1, 0, 0, 2, 0]
[2, 0, 0, 2, 0]
[0, 1, 0, 2, 0]
[1, 1, 0, 2, 0]
[2, 1, 0, 2, 0]
[0, 2, 0, 2, 0]
[1, 2, 0, 2, 0]
[2, 2, 0